## Win Prediction 2
Combining synergy of pairs of heroes into consideration

In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier

import seaborn as sbn
import psycopg2
import itertools
from matplotlib import pyplot

%matplotlib inline

In [2]:
##Functinons
def unstack_simplify(df):
    return df.unstack().iloc[10:20].reset_index(drop=True)

In [3]:
#Postgres connection
hostname = 'localhost'
username = 'postgres'
password = ''
database = 'dota2_data'


In [4]:
#get data from db and store into df

myConnection = psycopg2.connect(host=hostname, user=username, password=password, dbname=database)
cur = myConnection.cursor()
matches_df = pd.read_sql_query("select * from matches", myConnection)
players_df = pd.read_sql_query("select match_id, hero_id, account_id, player_slot from player_matches", myConnection)
heros_df = pd.read_sql_query("select * from heroes", myConnection)

In [5]:
ids = players_df['match_id'].values[0:10]
matches_df[matches_df['match_id'].isin(ids)]

,match_id,match_seq_num,radiant_win,start_time,duration,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire,cluster,...,dire_team_complete,radiant_captain,dire_captain,chat,objectives,radiant_gold_adv,radiant_xp_adv,teamfights,version,cosmetics
10970,2521743304,None,False,1469221556,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
28279,1174950966,None,False,1421696088,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
29358,1106986271,None,False,1419179985,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
32473,956361773,None,False,1413117649,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
35716,755072849,None,True,1404353703,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
40299,589126704,None,False,1396292356,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [6]:
heros_df[heros_df.id==114]

,id,name,localized_name,primary_attr,attack_type,roles
103,114,npc_dota_hero_monkey_king,Monkey King,agi,Melee,"[Carry, Escape, Disabler, Initiator]"


In [7]:
players_df.sort_values('match_id').head(10)

,match_id,hero_id,account_id,player_slot
307659,17955123,21,86747757,131
307660,17955123,33,88792641,132
307658,17955123,28,88704095,130
248399,17955123,9,69325073,128
307657,17955123,79,58017868,129
248394,17955123,31,89782335,0
248396,17955123,29,92926005,2
248395,17955123,32,82262664,1
248398,17955123,52,93712171,4
248397,17955123,39,85815961,3


In [8]:
test = players_df[['match_id','hero_id']].groupby('match_id').agg('count').sum(axis=1)
bad_matches = test[test!=10].index
players_df = players_df[~players_df['match_id'].isin(bad_matches)]

In [9]:
#match_hero_picks = players_df.groupby('match_id').apply(unstack_simplify)
#match_hero_picks.to_csv("match_hero_picks.csv")

In [10]:
#check data to make sure there are 10 heros in each row
match_hero_picks = pd.read_csv("match_hero_picks.csv")
test = match_hero_picks.groupby(['match_id']).agg('count').sum(axis=1)
test[test!=10]

Series([], dtype: int64)

In [11]:
#get synergy_data to add to match_hero_picks
synergy_matrix = np.loadtxt('synergy_matrix.csv',delimiter=",")
synergy_matrix

array([[ 0.        ,  0.48591549,  0.51428571, ...,  0.55633803,
         1.        ,  0.31578947],
       [ 0.48591549,  0.        ,  0.55434783, ...,  0.43518519,
         0.5       ,  0.49019608],
       [ 0.51428571,  0.55434783,  0.        , ...,  0.45098039,
         0.        ,  0.        ],
       ..., 
       [ 0.55633803,  0.43518519,  0.45098039, ...,  0.        ,
         0.        ,  0.66666667],
       [ 1.        ,  0.5       ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.31578947,  0.49019608,  0.        , ...,  0.66666667,
         0.        ,  0.        ]])

In [12]:
synergy_matrix[synergy_matrix == 1.0] = 0
synergy_matrix.max()


0.8571428571428571

In [13]:
matches_df[matches_df['match_id']==17955123]['radiant_win']

Series([], Name: radiant_win, dtype: bool)

In [15]:
#make feature vector
#index 0-113 is radiant heroes, 115-228 is dire
feature_list = []

feature_list_without_synergy = []

outcome = []

for row in match_hero_picks.iterrows():
    rad_synergy = 0
    dire_synergy = 0
    #monkey king update
    #228 hero vectors and 2 for synergy(114 and 229)
    vec = [0]*230
    vec_no_synergy = [0]*228
    #get match id to create list of win/loss
    match_id = row[1]['match_id']
    rad_win = matches_df[matches_df['match_id']==match_id]['radiant_win']
    if len(rad_win)<1:
        continue
    outcome.append(rad_win.values[0])
    #create the vector of length 226 and populate accordingly
    radiant_hero_picks = row[1][1:6]
    dire_hero_picks = row[1][6:11]

        
    for k,v in itertools.combinations(radiant_hero_picks.values,2):
        rad_synergy += synergy_matrix[k-1][v-1]
    vec[114]=rad_synergy/7

    for k,v in itertools.combinations(dire_hero_picks.values,2):
        dire_synergy += synergy_matrix[k-1][v-1]
    vec[229]=dire_synergy/7   
    
    for value in radiant_hero_picks.values:
        try:
            vec[value-1] = 1
        except:
            print value, ':', row
        
    for value in dire_hero_picks.values:
        try:
            vec[value+114] = 1
        except:
            print value, ':', row
        
    feature_list.append(vec)
feature_df = pd.DataFrame(feature_list)

In [16]:
feature_df.values.max()

1.0

In [17]:
len(outcome), feature_df.shape

(48367, (48367, 230))

In [18]:
in_train, in_test, out_train,  out_test = train_test_split(feature_df, outcome, test_size=0.33)


In [19]:
#create model and train
logreg = linear_model.LogisticRegression()
logreg.fit(in_train, out_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [20]:
output = logreg.predict(in_test)
error = np.mean(output != out_test)
print error

0.45645909034


In [21]:
rfc = RandomForestClassifier()
rfc.fit(in_train, out_train)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [22]:
output = rfc.predict(in_test)
error = np.mean(output != out_test)
print error

0.486530509961


In [ ]:
#need to check if training error got better because testing error did not change much